In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [2]:
# load all models in \cs_mods\July2023 folder by cycling through names
from cobra.io import read_sbml_model    
from pathlib import Path
import pandas as pd

# do not change directory but still go to the folder with the models
folder = Path().cwd() / "cs_mods" / "July2023"
model_list = []
for file in folder.glob("*.xml"):  
    model = read_sbml_model(file)
    model.id = file.stem # remove .xml 
    model.name = file.stem # remove .xml
    model_list.append(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized


In [3]:
# load old combined RPE_PR models
mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR_old'
mod_RPE_PR.name = 'RPE_PR_old'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod_Human1_Human1.id = 'RPE_PR_control_old'
mod_Human1_Human1.name = 'RPE_PR_control_old'
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')
mod.id = 'Human1_old'
mod.name = 'Human1_old'

# add old models to model_list
model_list = model_list + [mod,mod_RPE_PR]

# check out model_list
model_list

[<Model Human-GEM_28062023 at 0x2015a6c54e0>,
 <Model mod_RPE_PRcone_Liang at 0x20163276650>,
 <Model mod_RPE_PRcone_Liang_singlePR at 0x2012b529690>,
 <Model mod_RPE_PRcone_Liang_singleRPE at 0x2012f8d7a30>,
 <Model mod_RPE_PRcone_Lukowski at 0x2013e05a500>,
 <Model mod_RPE_PRcone_Lukowski_singlePR at 0x201454c8970>,
 <Model mod_RPE_PRcone_Lukowski_singleRPE at 0x20149825750>,
 <Model mod_RPE_PRcontrol at 0x20183125f60>,
 <Model mod_RPE_PRrod_Liang at 0x201991ae7a0>,
 <Model mod_RPE_PRrod_Liang_singlePR at 0x201e03fe590>,
 <Model mod_RPE_PRrod_Liang_singleRPE at 0x201e26ba770>,
 <Model mod_RPE_PRrod_Lukowski at 0x201e769fd60>,
 <Model mod_RPE_PRrod_Lukowski_singlePR at 0x201dfe37670>,
 <Model mod_RPE_PRrod_Lukowski_singleRPE at 0x201f2146b00>,
 <Model Human1_old at 0x2020e1796f0>,
 <Model RPE_PR_old at 0x201524b7a30>]

In [8]:
### COMPARE REACTIONS IN ROD MODELS

# get rod models
rod_Liang = [model for model in model_list if model.id == 'mod_RPE_PRrod_Liang_singlePR'][0]
rod_Lukowski = [model for model in model_list if model.id == 'mod_RPE_PRrod_Lukowski_singlePR'][0]
Human1 = [model for model in model_list if model.id == 'Human-GEM_28062023'][0]

# get reaction ids that are present in rod_Liang but not in rod_Lukowski and vice versa# get sets of reaction IDs for each model
liang_rxns = set(rxn.id for rxn in rod_Liang.reactions)
lukowski_rxns = set(rxn.id for rxn in rod_Lukowski.reactions)

# find reaction IDs that are present in Liang but not in Lukowski
liang_not_lukowski = liang_rxns - lukowski_rxns

# find reaction IDs that are present in Lukowski but not in Liang
lukowski_not_liang = lukowski_rxns - liang_rxns

# find reactions common to both models
common_rxns = liang_rxns.intersection(lukowski_rxns)

human1_rxns = set(rxn.id for rxn in Human1.reactions)

In [9]:
print('number of reactions in Human1: ' + str(len(human1_rxns)))
print('number of reactions in Liang rod AND in Lukowski rod: ' + str(len(common_rxns)))
print('number of reactions in Liang rod but not in Lukowski rod: ' + str(len(liang_not_lukowski)))
print('number of reactions in Lukowski rod but not in Liang rod: '+ str(len(lukowski_not_liang)))

number of reactions in Human1: 13085
number of reactions in Liang rod AND in Lukowski rod: 5599
number of reactions in Liang rod but not in Lukowski rod: 655
number of reactions in Lukowski rod but not in Liang rod: 861


In [5]:
# remove everyting after the first underscore in cell specific reaction IDs
liang_rxns_short = [rxn.split('_')[0] for rxn in liang_rxns]    
lukowski_rxns_short = [rxn.split('_')[0] for rxn in lukowski_rxns]

In [6]:
from src.get_info import make_rxn_df

# convert sets to lists
liang_not_lukowski_list = list(liang_not_lukowski)
lukowski_not_liang_list = list(lukowski_not_liang)
common_rxns_list = list(common_rxns)

# make rxn df of reactions liang_not_lukowski and lukowski_not_liang
liang_not_lukowski_rxns = make_rxn_df(rod_Liang).loc[liang_not_lukowski_list]
lukowski_not_liang_rxns = make_rxn_df(rod_Lukowski).loc[lukowski_not_liang_list]
common_rxns_rxns = make_rxn_df(rod_Liang).loc[common_rxns_list]

# write dfs to spreadsheet in results folder (results/compare_rod_models/), each df on different sheet (make folder if not exist)
from pathlib import Path

Path(Path().cwd() / 'results' / 'compare_rod_models').mkdir(parents=True, exist_ok=True)
with pd.ExcelWriter(Path().cwd() / 'results' / 'compare_rod_models' / 'compare_rod_models.xlsx') as writer:
    liang_not_lukowski_rxns.to_excel(writer, sheet_name = 'liang_not_lukowski')
    lukowski_not_liang_rxns.to_excel(writer, sheet_name = 'lukowski_not_liang')
    common_rxns_rxns.to_excel(writer, sheet_name = 'common_rxns')
    make_rxn_df(Human1).to_excel(writer, sheet_name = 'Human1')     # add Human1 df to spreadsheet

In [10]:
# modify models
# load blood exchange bounds

# for all models in the list set objective function (ATP hydrolysis, in PR for combined models)
for m in model_list:
    if 'MAR03964_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_PR'
    elif 'MAR03964_RPE' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_RPE'
    elif 'MAR03964' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964'
    else:
        print('no ATP hydrolysis reaction (MAR03964) in model: ' + m.id)

# list combined and single models
combined_models = [m for m in model_list if 'single' not in m.id and 'Human' not in m.id]
single_models = [m for m in model_list if 'single' in m.id]
Human1_models = [m for m in model_list if 'Human' in m.id]

# close PR exchange in combined models and open RPE exchange (efflux)
for m in combined_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,0)
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) 

# only allow efflux exchange in single models
for m in single_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,1000)

# for Human1 models, open efflux
for m in [m for m in model_list if 'Human' in m.id]:
    for r in [r for r in m.reactions if len(r.products) == 0]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000)



In [11]:
# set exchange bounds for blood
from src.modify_model import set_exchange_bounds
ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(7.25,1000)}
# MAR09048: oxygen
# MAR09034: glucose
# MAR09135: lactate

results_dict = {}
for m in model_list:
    set_exchange_bounds(m, ex_bounds)
    m.optimize()
    results_dict[m.id] = m.objective.value
df1 = pd.DataFrame(results_dict, index = ['max ATP hydrolysis (pmol/s/mm^2)']).T
df1

,max ATP hydrolysis (pmol/s/mm^2)
Human-GEM_28062023,24.7325
mod_RPE_PRcone_Liang,23.9000
mod_RPE_PRcone_Liang_singlePR,11.1350
mod_RPE_PRcone_Liang_singleRPE,23.9000
mod_RPE_PRcone_Lukowski,24.7325
mod_RPE_PRcone_Lukowski_singlePR,22.2350
mod_RPE_PRcone_Lukowski_singleRPE,23.9000
mod_RPE_PRcontrol,24.7325
mod_RPE_PRrod_Liang,23.9000
mod_RPE_PRrod_Liang_singlePR,13.9100


In [13]:
# FLUX VARIABILITY ANALYSIS

from cobra.flux_analysis import flux_variability_analysis

fba = {}
fva = {}
fva_loopless = {}

for m in [model_list[1]]:
    set_exchange_bounds(m, ex_bounds)
    m.optimize()
    fba[m.id] = m.objective.value
    fva[m.id] = flux_variability_analysis(m, loopless=False)
    fva_loopless[m.id] = flux_variability_analysis(m, loopless=True)

In [ ]:
# turn dicts into one df (add vertically)
df1 = pd.DataFrame(fba, index = ['FBA']).T
df2 = pd.DataFrame(fva).T
df3 = pd.DataFrame(fva_loopless).T

df2